In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
Items = pd.read_csv("../input/items-ff-final5a/Items_ff_final5.csv")
Items.head()
#Items.shape[0]

In [3]:
#Items.rename(columns={'Items1$total_sales': 'total_sales'}, inplace=True)
#Items.drop(columns=['description'], inplace=True)
Items['location'] = Items['location'].astype('category')
Items['condition'] = Items['condition'].astype('category')
Items['star_rating'] = Items['star_rating'].astype('category')
Items['revised_star_rating'] = Items['revised_star_rating'].astype('category')
Items['sales_cyber_monday'] = Items['sales_cyber_monday'].astype('category')
Items['sales_black_friday'] = Items['sales_black_friday'].astype('category')
Items['sales_colombus_day'] = Items['sales_colombus_day'].astype('category')
Items['sales_labor_day'] = Items['sales_labor_day'].astype('category')
Items['sales_presidents_day'] = Items['sales_presidents_day'].astype('category')
Items['sales_4th_july'] = Items['sales_4th_july'].astype('category')
Items['sales_newyear'] = Items['sales_newyear'].astype('category')
Items['sales_xms'] = Items['sales_xms'].astype('category')
Items['success'] = Items['success'].astype('category')
Items['brand_size'] = Items['brand_size'].astype('category')
Items['popular_brand'] = Items['popular_brand'].astype('category')
Items['category_size_rank_q'] = Items['category_size_rank_q'].astype('category')
Items['review_count_log'] = Items['review_count_log'].astype('category')
Items['price_diff_competitor_log'] = Items['price_diff_competitor_log'].astype('category')
Items['profit_pct_sqrt'] = Items['profit_pct_sqrt'].astype('category')
Items['reviews_cnt_category'] = Items['reviews_cnt_category'].astype('category')
Items['creation_time'] = Items['creation_time'].astype('datetime64')
Items.columns

In [4]:
#!pip install nltk
import nltk
nltk.download('punkt')

In [5]:
from nltk.tokenize import sent_tokenize

In [6]:
text = Items['description'].astype('str')
#text = text.sum()

In [7]:
text_df = pd.DataFrame()
text_df['description'] = text
text_df.head
text_df['char_count'] = text_df['description'].str.len() ## this also includes spaces
text_df[['description','char_count']].head()

In [8]:
# Average word length
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/len(words))

text_df['avg_word'] = text_df['description'].apply(lambda x: avg_word(x))
text_df[['description','avg_word']].head()

In [9]:
# Number of stop words
from nltk.corpus import stopwords
stop = stopwords.words('english')

text_df['stopwords'] = text_df['description'].apply(lambda x: len([x for x in x.split() if x in stop]))
text_df[['description','stopwords']].head()

In [10]:
# Number of numeric characters
text_df['numerics'] = text_df['description'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
text_df[['description','numerics']].head()

In [11]:
# Number of UPPER CASE words
text_df['upper'] = text_df['description'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
text_df[['description','upper']].head()

**Preprocessing**

In [12]:
# Lower case,remove stop words and Punctuation
from nltk.corpus import stopwords

text_df['text_df'] = text_df['description'].apply(lambda x: " ".join(x.lower() for x in x.split()))
text_df['text_df'] = text_df['text_df'].str.replace('[^\w\s]','')

stop = stopwords.words('english')
text_df['text_df'] = text_df['text_df'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
text_df['description'].head()

In [13]:
freq = pd.Series(' '.join(text_df['description']).split()).value_counts()[:10]
freq
freq = list(freq.index)
text_df['description'] = text_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
text_df['description'].head()

In [14]:
freq_rare = pd.Series(' '.join(text_df['description']).split()).value_counts()[-10:]
freq_rare
freq = list(freq_rare.index)
text_df['description'] = text_df['description'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_rare))
text_df['description'].head()

In [15]:
from textblob import TextBlob
text_df['description'][:5].apply(lambda x: str(TextBlob(x).correct()))


**Tokenization**

In [16]:
from textblob import TextBlob
TextBlob(text_df['description'][1]).words

In [17]:
from textblob import Word
text_df['description'] = text_df['description'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
text_df['description'].head()

**Sentiment Analysis**

In [18]:
text_df['sentiment'] = text_df['description'].apply(lambda x: TextBlob(x).sentiment[0] )
text_df[['description','sentiment']].head(10)

In [19]:
text_df.drop('description',axis=1, inplace = True)
text_df.to_csv("./text_analysis_2")